In [1]:
import requests
import time
import json
import os
import datetime
import boto3
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta

In [2]:
from config import ACCESS_KEY,SECRET_KEY,TOKEN,FOLDER,FOLDER_ID,BUCKET_NAME,BUCKET_NAME

In [22]:
TIME_ZONE = os.getenv("TIME_ZONE", "Europe/Moscow") #настройка функции
TEMP_FILENAME = "temp_file"

headers={'Authorization':TOKEN ,'Accept':'application/json'}

def get_now_datetime_str(): # получаем актуальное время
    time_zone = os.getenv("TIME_ZONE", "Europe/Moscow") # меняем таймзону на московскую
    now = datetime.datetime.now(pytz.timezone(time_zone))    
    yesterday = now - datetime.timedelta(days=1) #нужна вчерашняя дата так как данные за прошлый день
    last_month_data = now - relativedelta(month=1)
    return {'key_parquet': yesterday.strftime('year=%Y/month=%m/%d.parquet'),
            'key': yesterday.strftime('year=%Y/month=%m/%d.csv'),
            'key_month': yesterday.strftime('year=%Y/month=%m.csv'),
            'now':now.strftime('%Y-%m-%d %H:%M:%S'),
            'yesterday_data':yesterday.strftime('%Y-%m-%d'),
            'yesterday':yesterday.strftime('%Y-%m-%d %H:%M:%S'), 
            'year':yesterday.strftime('%Y'),
            'month':yesterday.strftime('%m'),
            'day':yesterday.strftime('%d'),
            'last_month_data':last_month_data.strftime('%Y-%m-%d')
            }

def create_query(): #функция создает новый запрос и возвращает id для запроса результата
    body = {
        "name":query_name, 
        "TYPE":"ANALYTICS", 
        "text":query_text, 
        "description":query_description
    }
    response = requests.post(
        f'https://api.yandex-query.cloud.yandex.net/api/fq/v1/queries?project={FOLDER_ID}',
        headers=headers,
        json=body
    )
    if response.status_code == 200:
        return response.json()["id"]
    return f' Code: {response},  text: {response.text}'


def get_request(offset): # фунция возвращает ответ запроса. Максимум 1000 строк.
    offset = offset
    get_query_results_url = f'https://api.yandex-query.cloud.yandex.net/api/fq/v1/queries/{request_id}/results/0?project={FOLDER_ID}&offset={str(offset)}&limit=1000'
    response = requests.get(
        get_query_results_url,
        headers = headers
    )
    return response

def if_cell_is_list(cell): # функция участвует в преобразовании данных при создании файла
    if isinstance(cell, list):
        if len(cell) == 0:
            return ''
        else: 
            return cell[0]
    else:
        return cell

def write_temp_file():
    offset = 0
    response = get_request(offset) #запрашиваем данные запроса
    columns = [rows['name'] for rows in response.json()['columns']] #выделяем названия столбцов
    special_str = ""
    for j in columns:
        special_str = f"{special_str}{str(j)},"
    temp_file = open(TEMP_FILENAME, 'w')
    temp_file.write(special_str[:-1]+'\n')

def write_temp_file():
    offset = 0
    response = get_request(offset) #запрашиваем данные запроса
    columns = [rows['name'] for rows in response.json()['columns']] #выделяем названия столбцов
    special_str = ""
    for j in columns:
        special_str = f"{special_str}{str(j)},"
    temp_file = open(TEMP_FILENAME, 'w', encoding='utf-8')
    temp_file.write(special_str[:-1]+'\n')

    while response.status_code == 200 and len(response.json()['rows']) != 0:  #Цикл делает запросы по 10000, пока не кончатся данные
        response = get_request(offset)
        response_rows = response.json()['rows']
        rows = [[if_cell_is_list(cell) for cell in row] for row in response_rows]  #Преобразуются строки
        # Открывает созданный файл и добавляет в него строки
        for row in rows:
            special_str = ','.join("'{0}'".format(i.replace("'", ""))  if isinstance(i, str) else str(i) for i in row)
            temp_file.write(special_str+'\n') 
        offset +=1000 # увеличивает смещение

def get_s3_instance(): # функция создает соединение
    session = boto3.session.Session()
    return session.client(
        aws_access_key_id=ACCESS_KEY,
        aws_secret_access_key=SECRET_KEY,
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net'
    )

def upload_dump_to_s3(): # функция выгружает данные в s3
    get_s3_instance().upload_file(
        Filename=TEMP_FILENAME,
        Bucket=BUCKET_NAME,
        Key=key
    )

def remove_temp_files(): #функция удаляет временный файл
    os.remove(TEMP_FILENAME)

In [23]:
s3 = get_s3_instance()

flats_st_partner

In [19]:
int(get_now_datetime_str()['year'])

'2024'

In [21]:
BUCKET_NAME = "dwh-asgard"
FOLDER = 'flats_st_partner'
key = f"year={int(get_now_datetime_str()['year'])}/month={int(get_now_datetime_str()['month'])}/{int(get_now_datetime_str()['day'])}.csv"
s3_file_name = f"{FOLDER}/{key}"
s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:3])

path_on_pc = 'E:/s3'
local_download_folder = f'{path_on_pc}/{s3_file_name_folder}'
local_download_path = f'{path_on_pc}/{s3_file_name}'

if os.path.isfile(local_download_path) is False:
# создается новая папка, если ее нет
    os.makedirs(local_download_folder, exist_ok=True)
    s3.download_file(Bucket=BUCKET_NAME,Key=s3_file_name,Filename=local_download_path)

In [7]:
flats_st_partner_df = pd.read_csv(f'{local_download_path}')

In [8]:
flats_st_partner_df

,report_date,flat_uuid,call_blocked,blocked,deleted
0,2024-04-18,f6c107ef-e972-4c14-ac9c-40b82b5f8395,0,0,0
1,2024-04-18,a498f0b2-b749-4255-9cf4-bf86663dfb60,0,0,0
2,2024-04-18,efc20b04-6a06-4ff1-9feb-27fce2279281,0,0,0
3,2024-04-18,c5ca5697-fae8-47f8-8ab1-34f1b11a9830,0,0,0
4,2024-04-18,4429f3bd-f810-4ef0-97dc-3ca2b7f7955f,0,0,0
...,...,...,...,...,...
714008,2024-04-18,a78c7a2a-708c-48db-98a3-14e76b3cfda7,0,0,0
714009,2024-04-18,18c5aaa1-254c-417f-99d3-a20107200440,0,0,0
714010,2024-04-18,347be418-1ee5-43b8-916a-cc6fec8e2103,0,0,0
714011,2024-04-18,c05b9086-8e3f-41c9-9707-fb9ce459030e,0,0,0


In [34]:
BUCKET_NAME = "dwh-asgard"
FOLDER = 'flats_dir_partner'
s3_file_name = f"{FOLDER}/flats_dir_partner.csv"
s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:1])

path_on_pc = 'E:/s3'
local_download_folder = f'{path_on_pc}/{s3_file_name_folder}'
local_download_path = f'{path_on_pc}/{s3_file_name}'

if os.path.isfile(local_download_path) is False:
# создается новая папка, если ее нет
    os.makedirs(local_download_folder, exist_ok=True)
    s3.download_file(Bucket=BUCKET_NAME,Key=s3_file_name,Filename=local_download_path)

In [12]:
flats_dir_partner_df = pd.read_csv(f'{local_download_path}')

In [13]:
flats_dir_partner_df

,created_at,number,address_uuid,installation_point_id,flat_uuid
0,2018-05-25 08:07:14,23,f59b9539-3cba-4818-af1c-b6b14e910db4,1665,f6c107ef-e972-4c14-ac9c-40b82b5f8395
1,2018-05-25 08:11:16,65,2c8ae56a-8938-4c9c-b3ac-a43caef5786e,1682,a498f0b2-b749-4255-9cf4-bf86663dfb60
2,2018-05-25 08:36:56,15,c711c9ff-c0ac-44c2-98e7-8c39ead87453,325,efc20b04-6a06-4ff1-9feb-27fce2279281
3,2018-05-25 08:39:02,2,501b7f66-eb3d-410f-96d0-da75fc9fdc0b,1667,c5ca5697-fae8-47f8-8ab1-34f1b11a9830
4,2018-05-25 08:40:27,25,501b7f66-eb3d-410f-96d0-da75fc9fdc0b,1670,4429f3bd-f810-4ef0-97dc-3ca2b7f7955f
...,...,...,...,...,...
714008,2024-04-19 02:17:48,101,d53755d5-afe6-4f9b-8d02-aa5e24fe88d4,748609,a78c7a2a-708c-48db-98a3-14e76b3cfda7
714009,2024-04-19 02:19:12,144,3b3255a5-f9bb-4788-a4aa-e26d70e82f4c,748610,18c5aaa1-254c-417f-99d3-a20107200440
714010,2024-04-19 02:37:03,104,f1b2bf33-b453-4169-9a3b-454466ddf43e,748611,347be418-1ee5-43b8-916a-cc6fec8e2103
714011,2024-04-19 02:37:30,33,08f672c3-0451-47b2-8d8a-2d56d031dc7f,748612,c05b9086-8e3f-41c9-9707-fb9ce459030e


In [14]:
flats_st_partner_df = flats_st_partner_df.merge(
    flats_dir_partner_df[['flat_uuid','address_uuid']],
    on='flat_uuid',
    how='left'
)

In [20]:
flats_st_partner_df['address_uuid'].nunique()

25264

installation_point_st_partner

In [25]:
BUCKET_NAME = "dwh-asgard"
FOLDER = 'installation_point_st_partner'
key = f"year={int(get_now_datetime_str()['year'])}/month={int(get_now_datetime_str()['month'])}/{int(get_now_datetime_str()['day'])}.csv"
s3_file_name = f"{FOLDER}/{key}"
s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:3])

path_on_pc = 'E:/s3'
local_download_folder = f'{path_on_pc}/{s3_file_name_folder}'
local_download_path = f'{path_on_pc}/{s3_file_name}'

if os.path.isfile(local_download_path) is False:
# создается новая папка, если ее нет
    os.makedirs(local_download_folder, exist_ok=True)
    s3.download_file(Bucket=BUCKET_NAME,Key=s3_file_name,Filename=local_download_path)

In [26]:
installation_point_st_partner_df = pd.read_csv(f'{local_download_path}')

In [28]:
installation_point_st_partner_df['installation_point_id'].nunique()

29704

In [29]:
BUCKET_NAME = "dwh-asgard"
FOLDER = 'entries_installation_points_dir_partner'
s3_file_name = f"{FOLDER}/entries_installation_points_dir_partner.csv"
s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:1])

path_on_pc = 'E:/s3'
local_download_folder = f'{path_on_pc}/{s3_file_name_folder}'
local_download_path = f'{path_on_pc}/{s3_file_name}'

if os.path.isfile(local_download_path) is False:
# создается новая папка, если ее нет
    os.makedirs(local_download_folder, exist_ok=True)
    s3.download_file(Bucket=BUCKET_NAME,Key=s3_file_name,Filename=local_download_path)

In [30]:
entries_installation_points_dir_partner_df = pd.read_csv(f'{local_download_path}')

In [33]:
installation_point_st_partner_df = installation_point_st_partner_df.merge(
    entries_installation_points_dir_partner_df[['installation_point_id','address_uuid','city','country','region','parent_uuid']],
    on='installation_point_id',
    how='left'
)

In [35]:
installation_point_st_partner_df['address_uuid'].nunique()

29704

entries_st_mobile

In [39]:
BUCKET_NAME = "dwh-asgard"
FOLDER = 'entries_st_mobile'
s3_file_name = f"{FOLDER}/{get_now_datetime_str()['key']}"
s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:3])

path_on_pc = 'E:/s3'
local_download_folder = f'{path_on_pc}/{s3_file_name_folder}'
local_download_path = f'{path_on_pc}/{s3_file_name}'

if os.path.isfile(local_download_path) is False:
# создается новая папка, если ее нет
    os.makedirs(local_download_folder, exist_ok=True)
    s3.download_file(Bucket=BUCKET_NAME,Key=s3_file_name,Filename=local_download_path)

In [41]:
entries_st_mobile_df = pd.read_csv(f'{local_download_path}')

In [42]:
entries_st_mobile_df

,report_date,address_uuid,partner_uuid,monetization
0,2024-04-18,000120a2-c280-41e5-817f-dd0834b3812c,71677fae-b464-435c-9916-206b0e9a92fe,True
1,2024-04-18,00044da9-eed9-4c0d-8514-75b7beef6f6d,NaN,True
2,2024-04-18,00057ce9-f508-490a-b154-cbb16cbb8257,db61f873-dcf7-4678-aec1-6b93f54ddbb2,False
3,2024-04-18,000c4f15-2478-4a10-a87e-b962c947ff53,0e7236ac-b8bd-4ab0-8634-d165ad17190e,False
4,2024-04-18,000c5c1a-b1bd-4b9c-9952-ee382c384130,a4cd78a8-0d04-4929-9add-08c793c8bc72,True
...,...,...,...,...
24540,2024-04-18,ffe99995-aef5-497e-8aea-0efb0eb9ef20,NaN,True
24541,2024-04-18,ffea0b88-8315-4ca7-9a44-af72cf04d53e,bec90e3e-2118-4766-b513-554d49e3c4d2,True
24542,2024-04-18,ffedcc55-182e-491e-943e-9cda52f41ecd,0e7236ac-b8bd-4ab0-8634-d165ad17190e,False
24543,2024-04-18,fff522f7-f0a9-4bb8-ba72-087ec4610a30,4900e775-a669-4624-b58d-916026610bba,True


merge

In [45]:
flats_st_partner_df_merged = flats_st_partner_df.merge(
    installation_point_st_partner_df[['installation_point_id','address_uuid','city','country','region','parent_uuid']],
    on='address_uuid',
    how='left'
)

flats_st_partner_df_merged=flats_st_partner_df_merged.merge(
    entries_st_mobile_df[['address_uuid','monetization']],
    on='address_uuid',
    how='left'
)

In [ ]:
BUCKET_NAME = "aggregated-data"
FOLDER = 'flats_research_dashboard'

local_download_folder= f"{path_on_pc}/{FOLDER}"
local_download_path = f"{path_on_pc}/{FOLDER}/{get_now_datetime_str()['key_parquet']}"

os.makedirs(local_download_folder, exist_ok=True)

In [53]:
flats_st_partner_df_merged.to_parquet(f"{path_on_pc}/{FOLDER}/{get_now_datetime_str()['key_parquet']}", compression='snappy', index=False) 

In [55]:
s3_file_path = f"{FOLDER}/{get_now_datetime_str()['key_parquet']}"
pc_file_path = f"{path_on_pc}/{FOLDER}/{get_now_datetime_str()['key_parquet']}"

s3.upload_file(pc_file_path, BUCKET_NAME, s3_file_path)